In [1]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
!pip install -q -U bitsandbytes


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

# Loading Dataset

In [2]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [3]:
!wandb login "ec4fd36593b5e8306001ca236dd6efc6424b46dc"

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import os
# from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient

# user_secrets = UserSecretsClient()

# login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

# os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "NLLB_2"
os.environ["WANDB_NOTES"] = "Fine tune nllb"
os.environ["WANDB_NAME"] = "finetune-NLLB-600M-on-opus100-Ar2En-without-optimization"

In [5]:
from datasets import get_dataset_config_names

configs=get_dataset_config_names("opus100")
print(configs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['af-en', 'am-en', 'an-en', 'ar-de', 'ar-en', 'ar-fr', 'ar-nl', 'ar-ru', 'ar-zh', 'as-en', 'az-en', 'be-en', 'bg-en', 'bn-en', 'br-en', 'bs-en', 'ca-en', 'cs-en', 'cy-en', 'da-en', 'de-en', 'de-fr', 'de-nl', 'de-ru', 'de-zh', 'dz-en', 'el-en', 'en-eo', 'en-es', 'en-et', 'en-eu', 'en-fa', 'en-fi', 'en-fr', 'en-fy', 'en-ga', 'en-gd', 'en-gl', 'en-gu', 'en-ha', 'en-he', 'en-hi', 'en-hr', 'en-hu', 'en-hy', 'en-id', 'en-ig', 'en-is', 'en-it', 'en-ja', 'en-ka', 'en-kk', 'en-km', 'en-kn', 'en-ko', 'en-ku', 'en-ky', 'en-li', 'en-lt', 'en-lv', 'en-mg', 'en-mk', 'en-ml', 'en-mn', 'en-mr', 'en-ms', 'en-mt', 'en-my', 'en-nb', 'en-ne', 'en-nl', 'en-nn', 'en-no', 'en-oc', 'en-or', 'en-pa', 'en-pl', 'en-ps', 'en-pt', 'en-ro', 'en-ru', 'en-rw', 'en-se', 'en-sh', 'en-si', 'en-sk', 'en-sl', 'en-sq', 'en-sr', 'en-sv', 'en-ta', 'en-te', 'en-tg', 'en-th', 'en-tk', 'en-tr', 'en-tt', 'en-ug', 'en-uk', 'en-ur', 'en-uz', 'en-vi', 'en-wa', 'en-xh', 'en-yi', 'en-yo', 'en-zh', 'en-zu', 'fr-nl', 'fr-ru', 'fr-zh', 

In [6]:
from datasets import load_dataset

dataset=load_dataset("opus100", "ar-en")
dataset

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

# Data Tokenization

In [7]:
from transformers import NllbTokenizerFast
import torch
model_name="facebook/nllb-200-distilled-600M"

tokenizer = NllbTokenizerFast.from_pretrained(
        "facebook/nllb-200-distilled-600M", src_lang="Arabic", tgt_lang="English")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [8]:
# use a sample of around 2000 instead of the complete dataset as training dataset
train_dataset=dataset['train'].shuffle(seed=42).select(range(8000))

# as evaluation dataset
eval_dataset=dataset['validation'].shuffle(seed=42).select(range(1600))

test_dataset=dataset['test'].shuffle(seed=42).select(range(1600))

def preprocess_func(data):
    inputs=[ex['ar'] for ex in data['translation']]
    targets=[ex['en'] for ex in data['translation']]
    
    # tokenize each row of inputs and outputs
    model_inputs=tokenizer(inputs, truncation=True)
    labels=tokenizer(targets, truncation=True)
    
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs


# We tokenize the entire dataset  

train_dataset=train_dataset.map(preprocess_func, batched=True)
eval_dataset=eval_dataset.map(preprocess_func, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# BLEU score

In [9]:
import evaluate
import numpy as np
import sacrebleu

metric_bleu = evaluate.load("sacrebleu")
metric_rouge = evaluate.load("rouge")

# Define a function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# Define a function to compute metrics
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    result_bleu = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    result_bleu = {"bleu": result_bleu["score"]}


    # Compute ROUGE score
    result_rouge = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels)
    result_rouge = {"rouge": result_rouge["rougeL"]}

    # Combine BLEU and ROUGE results
    result = {**result_bleu, **result_rouge}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    # Save the result to a file after each evaluation
    with open('metrics.txt', 'a') as f:
        f.write(f"BLEU: {result['bleu']}, ROUGE: {result['rouge']}, Gen Len: {result['gen_len']}\n")

    return result


# Training

In [10]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model=AutoModelForSeq2SeqLM.from_pretrained(model_name, use_cache=False)
print(model.config)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

M2M100Config {
  "_name_or_path": "facebook/nllb-200-distilled-600M",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "M2M100ForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 200,
  "max_position_embeddings": 1024,
  "model_type": "m2m_100",
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": true,
  "tokenizer_class": "NllbTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.42.4",
  "use_cache": false,
  "vocab_size": 256206
}



In [12]:
model

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

In [13]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available

True


In [14]:
!export CUDA_LAUNCH_BLOCKING=1

In [15]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [17]:
!pip install shutil

ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [18]:
import shutil

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [21]:
# Read the final BLEU score from the saved metrics file
with open('metrics.txt', 'r') as f:
    lines = f.readlines()
    print(lines)
# Print the last line which contains the final BLEU score
if lines:
    final_metrics = lines[-1]
    print("Final Evaluation Metrics:", final_metrics)
else:
    print("No evaluation metrics found.")


['BLEU: 33.9446, ROUGE: 0.6006, Gen Len: 17.7415\n', 'BLEU: 34.5466, ROUGE: 0.602, Gen Len: 17.6765\n', 'BLEU: 34.245, ROUGE: 0.6011, Gen Len: 17.6455\n']
Final Evaluation Metrics: BLEU: 34.245, ROUGE: 0.6011, Gen Len: 17.6455



In [22]:
import math

eval_results=trainer.evaluate()
print(eval_results)
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 3.36


In [ ]:
tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(os.getenv("WANDB_NAME"))

# Inference

In [24]:
# from transformers import pipeline

# text=" المملكة العربية السعودية"
# translator=pipeline("translation",model=os.getenv("WANDB_NAME"))
# translator(text)
model.config.use_cache=True
context=tokenizer(["ألخَيْـلُ وَاللّيْـلُ وَالبَيْـداءُ تَعرِفُنـي وَالسّيفُ وَالرّمحُ والقرْطاسُ وَالقَلَـمُ"], return_tensors="pt")
output=model.generate(**context)

tokenizer.decode(output[0], skip_special_tokens=True)

AttributeError: 'list' object has no attribute 'shape'

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set model configuration
model.config.use_cache = True

# Tokenize input text
context = tokenizer(["ألخَيْـلُ وَاللّيْـلُ وَالبَيْـداءُ تَعرِفُنـي وَالسّيفُ وَالرّمحُ والقرْطاسُ وَالقَلَـمُ"], return_tensors="pt")

# Move context tensors to the same device as the model
context = {key: value.to(device) for key, value in context.items()}

# Generate output
output = model.generate(input_ids=context["input_ids"], attention_mask=context["attention_mask"])

# Decode the generated output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Horse, night, plague, sword, spear, cross and word.
